# First Project of Oreilly Book

In [1]:
# os library

#checking if directory exists
#os.path.isdir(HOUSING_PATH)

#make dir

In [2]:
# Import cell

import os
import tarfile
import matplotlib.pyplot as plt
from six.moves import urllib
import pandas as pd

In [3]:
# Variable

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = "datasets/housing"
HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + "/housing.tgz"

In [4]:
# Functions

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
         os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [5]:
# Fetch and Load of datas

fetch_housing_data()
housing_data = load_housing_data()

In [6]:
# data exploration

housing_data.head(5)
housing_data.info()
housing_data['ocean_proximity'].value_counts()
housing_data.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20433 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [7]:
# Matplot Lib ipython only
plt.interactive(True)

housing_data.hist(bins=50, figsize=(20,15))
plt.show()

In [8]:
import numpy as np
import hashlib

def test_set_check(identifier, test_ratio, hash):
    hash(np.int64(identifier)).digest()[-1] < test_ratio * 256

def split_train_test(data, test_ratio):
    #create an array randomly permuted
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = len(data) * test_ratio
    tested_indices = shuffled_indices[:test_set_size]
    trained_indices = shuffled_indices[test_set_size:]
    return tested_indices, trained_indices

#version 2 with stable trained / tested sets
def split_train_test_v2(data, test_ratio, id_columns, hash=hashlib.md5):
    ids = data[id_columns]
    in_test_set=ids.apply(lambda id_ : hash(np.int64(id_)).digest()[-1] < test_ratio * 256)
    return data.loc[in_test_set], data.loc[~in_test_set]



In [9]:
#sckikit-learn alternative to function
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing_data,test_size=0.2, random_state=42)

In [10]:
hist?

In [11]:
housing_data_index = housing_data.reset_index()
housing_data_index.head()
    


,index,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [12]:
test_data,train_data = split_train_test_v2(housing_data_index, 0.2, 'index')
plt.close()
housing_data_index['median_income'].hist(rwidth=0.5, range=(0,20), bins=20, align='left')
plt.show()

In [13]:
housing_data_index['income_cat'] = np.ceil(housing_data_index['median_income']/1.5)
housing_data_index['income_cat'].where(housing_data_index['income_cat'] <5, 5.0, inplace=True)
plt.interactive(True)
plt.close()
housing_data_index['median_income'].hist(rwidth=0.5, range=(0,5), bins=6)
plt.show()
plt.clf



<function matplotlib.pyplot.clf>

In [14]:
housing_data_index["income_cat"].value_counts() / len (housing_data_index["income_cat"])

test_set['income_cat']= np.ceil(test_set['median_income']/1.5)
test_set['income_cat'].where(test_set['income_cat'] < 5, 5, inplace=True)

/home/mathieu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/mathieu/ml_mathieu/env/lib/python3.5/site-packages/pandas/core/generic.py:5233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [15]:
from sklearn.model_selection import StratifiedShuffleSplit
#Why using stratified
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(housing_data_index, housing_data_index['income_cat']):
    train_set = housing_data_index.loc[train_index]
    test_set = housing_data_index.loc[test_index]
    
print(train_set['income_cat'].value_counts() / len(train_set))
print(test_set['income_cat'].value_counts() / len(test_set))



3.0    0.350594
2.0    0.318859
4.0    0.176296
5.0    0.114402
1.0    0.039850
Name: income_cat, dtype: float64
3.0    0.350533
2.0    0.318798
4.0    0.176357
5.0    0.114583
1.0    0.039729
Name: income_cat, dtype: float64


In [16]:
housing = train_set.copy()

housing.info()

#here alpha allows to visualize high density data points
#housing.plot(kind="scatter", x="longitude", y="latitude", alpha="0.1")

housing.plot(kind="scatter", x="longitude", y="latitude", s=housing["population"]/100, label="population",
            c=housing["median_house_value"], cmap =plt.get_cmap("jet"))

plt.legend()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 17606 to 15775
Data columns (total 12 columns):
index                 16512 non-null int64
longitude             16512 non-null float64
latitude              16512 non-null float64
housing_median_age    16512 non-null float64
total_rooms           16512 non-null float64
total_bedrooms        16354 non-null float64
population            16512 non-null float64
households            16512 non-null float64
median_income         16512 non-null float64
median_house_value    16512 non-null float64
ocean_proximity       16512 non-null object
income_cat            16512 non-null float64
dtypes: float64(10), int64(1), object(1)
memory usage: 1.6+ MB


In [17]:
from pandas.tools.plotting import scatter_matrix

corr_matrix = housing.corr()

corr_matrix["total_bedrooms"].sort_values(ascending=False)

# pandas matrix
attributes=["median_income","median_house_value", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes])

/home/mathieu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: FutureWarning: 'pandas.tools.plotting.scatter_matrix' is deprecated, import 'pandas.plotting.scatter_matrix' instead.


array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7f54dd6cbc18>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x7f54d08574a8>]], dtype=object)

In [18]:
# creating more adapted parameters

housing["room_per_household"] = housing["total_rooms"] / housing["households"]
housing["bedroom_per_room"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["room_per_population"] = housing["total_rooms"] / housing["population"]

corr_matrix = housing.corr()

corr_matrix["median_house_value"].sort_values(ascending=False)

median_house_value     1.000000
median_income          0.687160
income_cat             0.642274
room_per_population    0.199429
room_per_household     0.146285
total_rooms            0.135097
housing_median_age     0.114110
index                  0.067723
households             0.064506
total_bedrooms         0.047689
population            -0.026920
longitude             -0.047432
latitude              -0.142724
bedroom_per_room      -0.259984
Name: median_house_value, dtype: float64

## Preparing clean data

In [19]:
#copying train_set again

housing = train_set.copy()
housing_labels = train_set["median_house_value"].copy()

housing.dropna(subset=["total_bedrooms"])
#housing.drop("total_bedrooms", axis=1)

median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median)

17606     351.0
18632     108.0
14650     471.0
3230      371.0
3555     1525.0
19480     588.0
8879      317.0
13685     293.0
4937      465.0
4861      229.0
16365     951.0
19684     559.0
19234     501.0
13956     582.0
2390      495.0
11176     649.0
15614     545.0
2953      251.0
13209     409.0
6569      261.0
5825      913.0
18086     538.0
16718     945.0
13600     278.0
13989     444.0
15168     190.0
6747      563.0
7398      366.0
5562      133.0
16121     416.0
          ...  
12380     767.0
5618       24.0
10060     539.0
18067     438.0
4471      797.0
19786     300.0
9969      393.0
14621    1051.0
579       302.0
11682    1615.0
245       460.0
12130     537.0
16441     544.0
11016     428.0
19934     422.0
1364       34.0
1236      829.0
5364      272.0
11703     300.0
10356     449.0
15270     515.0
3754      373.0
12166     756.0
6003      932.0
7364      212.0
6563      236.0
12053     294.0
13908     872.0
11159     380.0
15775     682.0
Name: total_bedrooms, Le

In [20]:
from sklearn.preprocessing import Imputer

housing = train_set.copy()
housing.drop("median_house_value", axis=1)
imputer = Imputer(strategy="median")
housing_num= housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)

print(imputer.statistics_)
housing_num.median().values

X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns=housing_num.columns)

[  1.03410000e+04  -1.18510000e+02   3.42600000e+01   2.90000000e+01
   2.11950000e+03   4.33000000e+02   1.16400000e+03   4.08000000e+02
   3.54090000e+00   1.79500000e+05   3.00000000e+00]


In [21]:
#LabelEncoder : numerical values for each label
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
housing_cat = housing["ocean_proximity"]
housing_cat_encoded = encoder.fit_transform(housing_cat)
housing_cat_encoded

encoder.classes_

array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'], dtype=object)

In [22]:
#OneHotEncoder : create vectors from a Labelencoder
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()

housing_cat_encoded_onehot = encoder.fit_transform(housing_cat_encoded.reshape(-1,1))
housing_cat_encoded_onehot
#output is a sparse matrix, needs to be turned into array
housing_cat_encoded_onehot.toarray()

array([[ 1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       ..., 
       [ 0.,  1.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.]])

In [23]:
#output is as we process one after the other transformer
from sklearn.preprocessing import LabelBinarizer

labelbinarizer = LabelBinarizer()
housing_cat_vector = labelbinarizer.fit_transform(housing_cat)
housing_cat_vector

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       ..., 
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0]])

In [24]:
#custom transformers

from sklearn.base import BaseEstimator, TransformerMixin

rooms_idx, bedrooms_idx, population_idx, household_idx = 3, 4, 5, 6 

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        return self # nothing to do
    
    def transform(self, X, y=None):
        rooms_per_household = X[:,rooms_idx] / X[:,household_idx]
        population_per_household = X[:,population_idx] / X[:,household_idx]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:,bedrooms_idx] / X[:,rooms_idx]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)


class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names].values
    
class LabelBinarizerPip(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return LabelBinarizer().fit(X).transform(X)

a = LabelBinarizerPip()
a.transform(housing_cat)

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       ..., 
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0]])

# Create Pipeline

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion

housing = train_set.copy()
housing = housing.drop("median_house_value", axis=1)
print(housing.head())
housing_num= housing.drop("ocean_proximity", axis=1)
num_attributes = list(housing_num)
cat_attributes = ["ocean_proximity"]

num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attributes)),
        ('imputer', Imputer(strategy='median')),
        ('attribut_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler())
    ])

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attributes)),
        ('label_binazer',LabelBinarizerPip()),
    ])


full_pipeline = FeatureUnion(transformer_list=[('num_pipeline', num_pipeline)
                                               , ('cat_pipeline', cat_pipeline)
                                              ])

housing_p = full_pipeline.fit_transform(housing)

#housing_tr = cat_pipeline.fit_transform(housing)

       index  longitude  latitude  housing_median_age  total_rooms  \
17606  17606    -121.89     37.29                38.0       1568.0   
18632  18632    -121.93     37.05                14.0        679.0   
14650  14650    -117.20     32.77                31.0       1952.0   
3230    3230    -119.61     36.31                25.0       1847.0   
3555    3555    -118.59     34.23                17.0       6592.0   

       total_bedrooms  population  households  median_income ocean_proximity  \
17606           351.0       710.0       339.0         2.7042       <1H OCEAN   
18632           108.0       306.0       113.0         6.4214       <1H OCEAN   
14650           471.0       936.0       462.0         2.8621      NEAR OCEAN   
3230            371.0      1460.0       353.0         1.8839          INLAND   
3555           1525.0      4459.0      1463.0         3.0347       <1H OCEAN   

       income_cat  
17606         2.0  
18632         5.0  
14650         2.0  
3230          2.0 

# Select and Train Models

In [26]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


lin_reg = LinearRegression()
lin_reg.fit(housing_p, housing_labels)

housing_predictions = lin_reg.predict(housing_p)
mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(mse)

print(lin_rmse)

68161.1001914


In [27]:
some_data = housing

some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
pred = lin_reg.predict(some_data_prepared)
print("Predictions:\t", lin_reg.predict(some_data_prepared))
print("Labels:\t\t", list(some_labels))

Predictions:	 [ 213836.02841195  331467.6533871   209832.27869359 ...,  101444.72769619
  215521.7803334   288527.55770957]
Labels:		 [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


In [28]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor()
tree.fit(housing_p, housing_labels)

tree_pred = tree.predict(housing_p)

treen_mse = mean_squared_error(housing_labels, tree_pred)
print(treen_mse)

0.0


# Cross Validation

In [35]:
from sklearn.model_selection import cross_val_score

tree_scores = cross_val_score(tree, housing_p, housing_labels, scoring="neg_mean_squared_error", cv=10)

tree_mse_scores = np.sqrt(-tree_scores)
print(tree_mse_scores)

tree_scores = cross_val_score(tree, housing_p, housing_labels, scoring="neg_mean_squared_error", cv=10)

tree_mse_scores = np.sqrt(-tree_scores)
print(tree_mse_scores)
print("Mean", tree_mse_scores.mean())
print("Std dev", reg_mse_scores.std())

[ 65096.10033421  65633.67814581  69474.32657272  69908.60231108
  70782.53236749  74281.83514172  64844.65213224  66830.27902154
  69480.06929236  72551.05274181]
[ 66087.36201845  66370.49431077  69407.11653159  70288.13918893
  72289.66065737  74493.59688772  65097.98328967  67274.53968569
  69092.16488677  72286.73530508]
Mean 69268.7792762
Std dev 2207.02667964


In [34]:
reg_scores = cross_val_score(lin_reg, housing_p, housing_labels, scoring="neg_mean_squared_error", cv=10)
reg_mse_scores = np.sqrt(-reg_scores)
print(reg_mse_scores)
print("Mean", reg_mse_scores.mean())
print("Std dev", reg_mse_scores.std())

[ 66783.86362957  66303.03098449  69294.17251515  72314.00837111
  68374.97616628  70936.51195447  64824.89685129  67762.31895378
  70630.50710503  67389.32822175]
Mean 68461.3614753
Std dev 2207.02667964


## Save model to come back

This is pretty useful to dump and load model

In [40]:
from sklearn.externals import joblib

joblib.dump(tree, "tree.pkl")

tree_loaded = joblib.load("tree.pkl")

None
